# Knowledge Distillation for EfficientNetV2 on CIFAR-100 (SOTA) - Local Version

This notebook implements **State-of-the-Art** knowledge distillation with:

- **Decoupled Knowledge Distillation (DKD)** - Separates target and non-target class knowledge
- **CutMix + Mixup** - Advanced data augmentation

---

## Requirements

```bash
pip install torch torchvision tqdm thop matplotlib numpy
```

### Expected Results:

- **Teacher (EfficientNetV2-L):** ~68-70% accuracy
- **Distilled Student (DKD):** ~69-72% accuracy

---


In [1]:
# Cell 1: Setup and Imports
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import efficientnet_v2_s, efficientnet_v2_l, EfficientNet_V2_S_Weights, EfficientNet_V2_L_Weights
from tqdm import tqdm
import copy
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
from pathlib import Path

# Setup Directories (Local paths)
PROJECT_ROOT = Path('./outputs')
MODEL_DIR = PROJECT_ROOT / 'models'
DATA_DIR = PROJECT_ROOT / 'data'
CHECKPOINT_DIR = PROJECT_ROOT / 'checkpoints'

MODEL_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directories ready at: {PROJECT_ROOT.absolute()}")

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Directories ready at: d:\Projects\MasterProject\code\outputs
Using device: cuda
GPU: NVIDIA GeForce RTX 5070 Laptop GPU


In [2]:
# Cell 2: Experiment Configuration (SOTA)
# ==========================================
# HYPERPARAMETERS
# ==========================================
NUM_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.05
PATIENCE = 30

# Distillation Params (DKD)
DKD_ALPHA = 1.0             # Weight for Target Knowledge
DKD_BETA = 8.0              # Weight for Non-Target Knowledge (Crucial)
TEMPERATURE = 4.0           # Softmax Temperature

# Augmentation Params
MIXUP_ALPHA = 0.8
CUTMIX_ALPHA = 1.0
CHECKPOINT_FREQUENCY = 20
NUM_CLASSES = 100

print(f"{'='*50}")
print(f"CONFIG: Epochs={NUM_EPOCHS} | Batch={BATCH_SIZE} | Temp={TEMPERATURE}")
print(f"DKD: Alpha={DKD_ALPHA}, Beta={DKD_BETA}")
print(f"{'='*50}")

CONFIG: Epochs=200 | Batch=128 | Temp=4.0
DKD: Alpha=1.0, Beta=8.0


In [3]:
# Cell 3: Data Loading (Enhanced with AutoAugment + RandomErasing)
from torchvision.transforms import autoaugment

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    autoaugment.AutoAugment(policy=autoaugment.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.2)),  # Cutout-like augmentation
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(root=str(DATA_DIR), train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, 
                                          num_workers=4, pin_memory=True, drop_last=True)

testset = torchvision.datasets.CIFAR100(root=str(DATA_DIR), train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, 
                                         num_workers=4, pin_memory=True)

print(f"Data loaded: {len(trainset)} Training, {len(testset)} Test images")
print(f"  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup")

Data loaded: 50000 Training, 10000 Test images
  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup


In [4]:
# Cell 4: Helper Functions (Enhanced KD Loss with Label Smoothing)

# ==========================================
# 1. Enhanced Knowledge Distillation Loss
# With Label Smoothing for better generalization
# ==========================================
def kd_loss(student_logits, teacher_logits, labels, temp=4.0, alpha=0.7, label_smoothing=0.1):
    """
    Enhanced Knowledge Distillation Loss
    - Soft loss: KL divergence between student and teacher soft targets
    - Hard loss: Cross-entropy with label smoothing
    - alpha: weight for soft loss (1-alpha for hard loss)
    """
    # Soft targets (KL Divergence)
    soft_student = F.log_softmax(student_logits / temp, dim=1)
    soft_teacher = F.softmax(teacher_logits / temp, dim=1)
    soft_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temp ** 2)
    
    # Hard targets with Label Smoothing
    hard_loss = F.cross_entropy(student_logits, labels, label_smoothing=label_smoothing)
    
    # Combined loss
    loss = alpha * soft_loss + (1 - alpha) * hard_loss
    
    return loss

def kd_loss_mixup(student_logits, teacher_logits, labels_a, labels_b, lam, 
                  temp=4.0, alpha=0.7, label_smoothing=0.1):
    """
    Enhanced KD Loss for mixed samples (CutMix/Mixup compatible)
    With Label Smoothing
    """
    # Soft targets (KL Divergence)
    soft_student = F.log_softmax(student_logits / temp, dim=1)
    soft_teacher = F.softmax(teacher_logits / temp, dim=1)
    soft_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temp ** 2)
    
    # Hard targets with Label Smoothing (Mixed)
    hard_loss = lam * F.cross_entropy(student_logits, labels_a, label_smoothing=label_smoothing) + \
                (1 - lam) * F.cross_entropy(student_logits, labels_b, label_smoothing=label_smoothing)
    
    # Combined loss
    loss = alpha * soft_loss + (1 - alpha) * hard_loss
    
    # NaN safety check
    if torch.isnan(loss):
        return hard_loss  # Fallback to CE only
    
    return loss

# ==========================================
# 2. Augmentations: Mixup & CutMix
# ==========================================
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

# ==========================================
# 3. Utilities (Save/Load/Evaluate)
# ==========================================
def evaluate_model_with_loss(model, dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    avg_loss = running_loss / len(dataloader)
    return acc, avg_loss

def save_checkpoint(model, optimizer, scheduler, epoch, best_acc, history, model_name, epochs_no_improve):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_acc': best_acc,
        'history': history,
        'epochs_no_improve': epochs_no_improve
    }
    path = CHECKPOINT_DIR / f"{model_name}_epoch{epoch+1}.pth"
    torch.save(checkpoint, path)
    print(f"  Checkpoint saved: {path}")
    
def load_checkpoint(model, optimizer, scheduler, model_name):
    checkpoints = sorted(glob.glob(str(CHECKPOINT_DIR / f"{model_name}_epoch*.pth")))
    if not checkpoints:
        return None
    latest = checkpoints[-1]
    print(f"  Loading checkpoint: {latest}")
    checkpoint = torch.load(latest, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint

def cleanup_old_checkpoints(model_name, keep=3):
    checkpoints = sorted(glob.glob(str(CHECKPOINT_DIR / f"{model_name}_epoch*.pth")))
    if len(checkpoints) > keep:
        for chk in checkpoints[:-keep]:
            os.remove(chk)
            print(f"  Cleaned up: {os.path.basename(chk)}")

print("Helper functions loaded (Enhanced: Label Smoothing + Optimized Alpha)")

Helper functions loaded (Enhanced: Label Smoothing + Optimized Alpha)


In [5]:
# Cell 5: Enhanced Training Loop (with LR Warmup)
def train_model_optimized(model, dataloader, optimizer, scheduler, num_epochs, model_name, 
                         teacher_model=None, temp=4.0, kd_alpha=0.7, label_smoothing=0.1,
                         patience=30, grad_clip=1.0, warmup_epochs=5):
    """
    Enhanced Training loop with:
    - Standard KD loss with Label Smoothing
    - CutMix/Mixup augmentation
    - Learning Rate Warmup
    - Mixed precision training
    - Gradient clipping
    - NaN detection and recovery
    """
    
    # 1. Load Checkpoint
    checkpoint = load_checkpoint(model, optimizer, scheduler, model_name)
    if checkpoint:
        start_epoch = checkpoint['epoch'] + 1
        best_acc = checkpoint['best_acc']
        history = checkpoint['history']
        epochs_no_improve = checkpoint['epochs_no_improve']
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  Resuming from epoch {start_epoch}, Best Acc: {best_acc:.2f}%")
    else:
        start_epoch = 0
        best_acc = 0.0
        history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}
        epochs_no_improve = 0
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  Starting fresh training...")

    # 2. Setup
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
    val_criterion = nn.CrossEntropyLoss()
    
    # Store base LR for warmup
    base_lr = optimizer.param_groups[0]['lr']
    
    if teacher_model:
        teacher_model.eval()
        for param in teacher_model.parameters():
            param.requires_grad = False

    # 3. Training Loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        valid_batches = 0
        
        # Learning Rate Warmup
        if epoch < warmup_epochs:
            warmup_lr = base_lr * (epoch + 1) / warmup_epochs
            for param_group in optimizer.param_groups:
                param_group['lr'] = warmup_lr
        
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Randomly choose Mixup (50%) or CutMix (50%)
            use_cutmix = np.random.rand() > 0.5
            if use_cutmix:
                inputs_aug, labels_a, labels_b, lam = cutmix_data(inputs.clone(), labels, alpha=CUTMIX_ALPHA)
            else:
                inputs_aug, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=MIXUP_ALPHA)
            
            optimizer.zero_grad()
            
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                # Student Forward
                student_outputs = model(inputs_aug)
                
                # Loss Calculation
                if teacher_model:
                    # Teacher Forward (no grad)
                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs_aug)
                    
                    # Enhanced KD Loss with Label Smoothing
                    loss = kd_loss_mixup(
                        student_outputs, teacher_outputs, 
                        labels_a, labels_b, lam,
                        temp=temp, alpha=kd_alpha, label_smoothing=label_smoothing
                    )
                else:
                    # Standard CE with Label Smoothing for Teacher training
                    loss = lam * F.cross_entropy(student_outputs, labels_a, label_smoothing=label_smoothing) + \
                           (1 - lam) * F.cross_entropy(student_outputs, labels_b, label_smoothing=label_smoothing)
            
            # Skip batch if loss is NaN
            if torch.isnan(loss):
                loop.set_postfix(loss="NaN-skip")
                continue
            
            # Backward
            scaler.scale(loss).backward()
            
            if grad_clip > 0:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            valid_batches += 1
            loop.set_postfix(loss=f"{loss.item():.3f}")

        # Step Scheduler (only after warmup)
        if epoch >= warmup_epochs:
            scheduler.step()
        
        # Validation
        train_loss = running_loss / max(valid_batches, 1)
        val_acc, val_loss = evaluate_model_with_loss(model, testloader, val_criterion)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)
        
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | LR: {current_lr:.6f}")
        
        # Save Best
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), MODEL_DIR / f"{model_name}.pth")
            epochs_no_improve = 0
            print(f"  New best model saved! Accuracy: {best_acc:.2f}%")
        else:
            epochs_no_improve += 1
            
        # Checkpointing
        if (epoch + 1) % CHECKPOINT_FREQUENCY == 0:
            save_checkpoint(model, optimizer, scheduler, epoch, best_acc, history, model_name, epochs_no_improve)
            cleanup_old_checkpoints(model_name)
            
        # Early Stopping
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    print(f"\nTraining complete. Best accuracy: {best_acc:.2f}%")
    model.load_state_dict(best_model_wts)
    return model, history

print("Training function loaded (Enhanced: LR Warmup + Label Smoothing)")

Training function loaded (Enhanced: LR Warmup + Label Smoothing)


In [6]:
# Cell 6: Initialize Models
print("Loading Teacher (EfficientNetV2-L)...")
teacher_model = efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1)
teacher_model.classifier[1] = nn.Linear(teacher_model.classifier[1].in_features, NUM_CLASSES)
teacher_model = teacher_model.to(device)

print("Loading Student (EfficientNetV2-S)...")
student_model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
student_model.classifier[1] = nn.Linear(student_model.classifier[1].in_features, NUM_CLASSES)
student_model = student_model.to(device)

print("Models loaded")

Loading Teacher (EfficientNetV2-L)...
Loading Student (EfficientNetV2-S)...
Models loaded


In [7]:
# Cell 7: Train/Load Teacher Model
print("\n" + "="*70)
print("TEACHER MODEL")
print("="*70)

teacher_path = MODEL_DIR / "teacher_model.pth"
if teacher_path.exists():
    print(f"Found existing Teacher Model: {teacher_path}")
    teacher_model.load_state_dict(torch.load(teacher_path, map_location=device))
else:
    print("Training Teacher Model (This may take a while)...")
    opt_t = optim.AdamW(teacher_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    sch_t = optim.lr_scheduler.CosineAnnealingLR(opt_t, T_max=NUM_EPOCHS)
    teacher_model, teacher_history = train_model_optimized(
        teacher_model, trainloader, opt_t, sch_t, NUM_EPOCHS, "teacher_model", teacher_model=None
    )

# Evaluate Teacher
teacher_model.eval()
teacher_accuracy, _ = evaluate_model_with_loss(teacher_model, testloader, nn.CrossEntropyLoss())
print(f"\nTeacher Accuracy: {teacher_accuracy:.2f}%")


TEACHER MODEL
Found existing Teacher Model: outputs\models\teacher_model.pth

Teacher Accuracy: 75.76%


In [8]:
# Cell 8: Train Enhanced Distilled Student
print("\n" + "="*70)
print("ENHANCED DISTILLED STUDENT (v2)")
print("="*70)

# Enhanced KD Parameters
KD_ALPHA = 0.7           # More hard label signal (was 0.9)
KD_TEMP = 4.0            # Temperature for softening
LABEL_SMOOTHING = 0.1    # Regularization
WARMUP_EPOCHS = 5        # LR warmup

student_name = "distilled_student_enhanced"
student_path = MODEL_DIR / f"{student_name}.pth"

if student_path.exists():
    print(f"Found existing Enhanced Model: {student_path}")
    student_model.load_state_dict(torch.load(student_path, map_location=device))
    distilled_accuracy, _ = evaluate_model_with_loss(student_model, testloader, nn.CrossEntropyLoss())
    print(f"Enhanced Student Accuracy: {distilled_accuracy:.2f}%")
else:
    print(f"\nStarting Enhanced Knowledge Distillation...")
    print(f"  KD Alpha (soft weight): {KD_ALPHA}")
    print(f"  Temperature: {KD_TEMP}")
    print(f"  Label Smoothing: {LABEL_SMOOTHING}")
    print(f"  LR Warmup: {WARMUP_EPOCHS} epochs")
    print(f"  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup")
    
    # Re-initialize student model (fresh weights from ImageNet)
    student_model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    student_model.classifier[1] = nn.Linear(student_model.classifier[1].in_features, NUM_CLASSES)
    student_model = student_model.to(device)
    
    # Optimizer & Scheduler
    opt_s = optim.AdamW(student_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    sch_s = optim.lr_scheduler.CosineAnnealingLR(opt_s, T_max=NUM_EPOCHS - WARMUP_EPOCHS)
    
    # Run Enhanced Training
    trained_student, distilled_history = train_model_optimized(
        model=student_model,
        dataloader=trainloader,
        optimizer=opt_s,
        scheduler=sch_s,
        num_epochs=NUM_EPOCHS,
        model_name=student_name,
        teacher_model=teacher_model,
        temp=KD_TEMP,
        kd_alpha=KD_ALPHA,
        label_smoothing=LABEL_SMOOTHING,
        patience=PATIENCE,
        warmup_epochs=WARMUP_EPOCHS
    )
    
    distilled_accuracy, _ = evaluate_model_with_loss(trained_student, testloader, nn.CrossEntropyLoss())
    print(f"\nEnhanced Student Final Accuracy: {distilled_accuracy:.2f}%")


ENHANCED DISTILLED STUDENT (v2)

Starting Enhanced Knowledge Distillation...
  KD Alpha (soft weight): 0.7
  Temperature: 4.0
  Label Smoothing: 0.1
  LR Warmup: 5 epochs
  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup
  Starting fresh training...


Epoch 1/200: 100%|██████████| 390/390 [01:05<00:00,  5.93it/s, loss=1.797]


Epoch 1/200 | Train Loss: 2.0196 | Val Loss: 2.9395 | Val Acc: 31.13% | LR: 0.000200
  New best model saved! Accuracy: 31.13%


Epoch 2/200: 100%|██████████| 390/390 [01:16<00:00,  5.11it/s, loss=1.795]


Epoch 2/200 | Train Loss: 1.7863 | Val Loss: 2.3039 | Val Acc: 43.64% | LR: 0.000400
  New best model saved! Accuracy: 43.64%


...

Epoch 198/200 | Train Loss: 1.1348 | Val Loss: 1.2664 | Val Acc: 73.54% | LR: 0.000000


Epoch 199/200: 100%|██████████| 390/390 [01:12<00:00,  5.40it/s, loss=0.923]


Epoch 199/200 | Train Loss: 1.1275 | Val Loss: 1.1881 | Val Acc: 74.16% | LR: 0.000000


Epoch 200/200: 100%|██████████| 390/390 [01:12<00:00,  5.40it/s, loss=1.131]


Epoch 200/200 | Train Loss: 1.1472 | Val Loss: 1.2012 | Val Acc: 74.03% | LR: 0.000000
  Checkpoint saved: outputs\checkpoints\distilled_student_enhanced_epoch200.pth
  Cleaned up: distilled_student_enhanced_epoch200.pth

Training complete. Best accuracy: 74.21%

Enhanced Student Final Accuracy: 74.21%


In [9]:
# Cell 9: Results Summary
print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)

print("\n| Model                      | Accuracy (%) |")
print("|----------------------------|--------------|")
try:
    print(f"| Teacher (EfficientNet-L)   | {teacher_accuracy:12.2f} |")
except NameError:
    print(f"| Teacher (EfficientNet-L)   | {'N/A':>12} |")

try:
    print(f"| Enhanced Student (v2)      | {distilled_accuracy:12.2f} |")
except NameError:
    print(f"| Enhanced Student (v2)      | {'N/A':>12} |")

try:
    improvement = distilled_accuracy - teacher_accuracy
    print(f"\n{'='*80}")
    if improvement > 0:
        print(f"Student SURPASSED Teacher by: {improvement:+.2f}%")
    else:
        print(f"Gap from Teacher: {improvement:.2f}%")
    print(f"{'='*80}")
except NameError:
    pass

print(f"\nEnhancements applied:")
print(f"  - AutoAugment + RandomErasing")
print(f"  - Label Smoothing (0.1)")
print(f"  - LR Warmup (5 epochs)")
print(f"  - Optimized KD Alpha (0.7)")

print(f"\nAll models saved to: {MODEL_DIR.absolute()}")
print(f"Checkpoints saved to: {CHECKPOINT_DIR.absolute()}")


FINAL RESULTS SUMMARY

| Model                      | Accuracy (%) |
|----------------------------|--------------|
| Teacher (EfficientNet-L)   |        75.76 |
| Enhanced Student (v2)      |        74.21 |

Gap from Teacher: -1.55%

Enhancements applied:
  - AutoAugment + RandomErasing
  - Label Smoothing (0.1)
  - LR Warmup (5 epochs)
  - Optimized KD Alpha (0.7)

All models saved to: d:\Projects\MasterProject\code\outputs\models
Checkpoints saved to: d:\Projects\MasterProject\code\outputs\checkpoints
